### Get the User's Desired LLM

In [1]:
import langchain_test as lt 
import methods

model = lt.get_user_choice()
print("You have chosen", model.name)

Please input your choice of LLM
Please enter a number
Please enter a number
Please enter a number
You have chosen ChatGPT


### Get User's Calendar

In [2]:
calendar = methods.get_calendar()
model.llm.invoke("Print my calendar for the week")
user_query = lt.get_user_query()

What change would you like to make to your calendar


### Set Up Prompts 

In [3]:
from langchain_core.prompts import ChatPromptTemplate

intent_prompt_template = ChatPromptTemplate.from_messages([
    ("human", "Identify the type of query that the following query is: {user_query}. Your response should be a single word that describes the intent of the query, and should be one of the following: createEvent, moveEvent, deleteEvent, checkSchedule, listEvents, updateEvent."),
])

update_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "append _format to the following user intent - {user_intent}"),
    ("system", "Only output the appended word. Your answer should be one word in the following format: userintent_format")
])

update_json_template = ChatPromptTemplate.from_messages([
    ("system", "You are a virtual calendar assistant helping a user update their calendar by formatting the update for Google Calendars"),
    ("human", "Can you make the following update to my calendar - {user_query}"),
    ("system", "Analyze the following calendar to understand the user's schedule - {user_calendar}"),
    ("system", "Ensure your output is a valid JSON Object. Here is guidance for your output - {format}. Ensure you follow the instructions in the description"),
])

### Build The Chain

In [4]:
from langchain_core.runnables import RunnablePassthrough
import formats

lcel_chain = intent_prompt_template | model.llm | {'user_intent':RunnablePassthrough()} |update_prompt_template | model.llm 
out = lcel_chain.invoke({"user_query":user_query})
command = out.content

lcel_chain_2 = update_json_template | model.llm 
out = lcel_chain_2.invoke({"user_query":user_query, "user_calendar":calendar, "time":methods.current_time, "format": formats.getFormat(command)[0]['parameters']['properties']})
print(out.content)

{
  "name": {
    "type": "string",
    "description": "Name of the event to be moved."
  },
  "date": {
    "type": "string",
    "description": "Current date of the event to be moved. Use YYYY-MM-DD format."
  },
  "time": {
    "type": "string",
    "description": "The time of the event to be moved. Use 12HR format, example: 10:00 AM."
  },
  "new_date": {
    "type": "string",
    "description": "The date to which the event is to be moved. Use YYYY-MM-DD format."
  },
  "new_time": {
    "type": "string",
    "description": "The time to which the event is to be moved. Use 12HR format, example: 10:00 AM."
  }
}


### Format JSON

In [5]:
import json

json_response = json.loads(out.content)
print(json_response)

methods.moveEvent(json_response)

{'name': {'type': 'string', 'description': 'Name of the event to be moved.'}, 'date': {'type': 'string', 'description': 'Current date of the event to be moved. Use YYYY-MM-DD format.'}, 'time': {'type': 'string', 'description': 'The time of the event to be moved. Use 12HR format, example: 10:00 AM.'}, 'new_date': {'type': 'string', 'description': 'The date to which the event is to be moved. Use YYYY-MM-DD format.'}, 'new_time': {'type': 'string', 'description': 'The time to which the event is to be moved. Use 12HR format, example: 10:00 AM.'}}


ValueError: time data "{'type': 'string', 'description': 'Current date of the event to be moved. Use YYYY-MM-DD format.'} {'type': 'string', 'description': 'The time of the event to be moved. Use 12HR format, example: 10:00 AM.'}" does not match format '%Y-%m-%d %I:%M %p'